# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from tokenz import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
# symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(api_url).json()
# data

symbol='aapl'
url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
outer_data = requests.get(url).json()
outer_data

[{'avgTotalVolume': 64010821,
  'calculationPrice': 'close',
  'change': 1.15,
  'changePercent': 0.00617,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexClose': 187.38,
  'iexCloseTime': 1715716799982,
  'iexLastUpdated': 1715716799982,
  'iexMarketPercent': 25.57895430579965,
  'iexOpen': 187.64,
  'iexOpenTime': 1715693400488,
  'iexRealtimePrice': 187.38,
  'iexRealtimeSize': 200,
  'iexVolume': 582177,
  'lastTradeTime': 1715716799983,
  'latestPrice': 187.43,
  'latestSource': 'Close',
  'latestTime': 'May 14, 2024',
  'latestUpdate': 1715716800171,
  'latestVolume': None,
  'low': None,
  'lowSource': None,


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
data = outer_data[0]
pe_ratio = data['peRatio']
pe_ratio

29.15

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [14]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker'][:100]:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    # batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    data = outer_data[0]
    new_row = pd.Series([symbol, 
                data['latestPrice'],
                data['peRatio'],
                'N/A'
                ], 
                index = my_columns)
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_14588\1178034005.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [15]:
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,151.28,36.11,N/A
1,AAL,15.19,23.17,N/A
2,AAP,75.64,151.64,N/A
3,AAPL,187.43,29.15,N/A
4,ABBV,161.59,48.09,N/A
...,...,...,...,...
95,CINF,118.21,7.87,N/A
96,CL,94.57,29.93,N/A
97,CLX,138.55,71.79,N/A
98,CMA,54.00,10.78,N/A


In [16]:
refrence_dataframe = final_dataframe

In [5]:
# # Function sourced from 
# # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]   
        
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
# #     print(symbol_strings[i])

# my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
# final_dataframe = pd.DataFrame(columns = my_columns)

# for symbol_string in symbol_strings:
# #     print(symbol_strings)
#     batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(batch_api_call_url).json()
#     for symbol in symbol_string.split(','):
#         final_dataframe = final_dataframe.append(
#                                         pd.Series([symbol, 
#                                                    data[symbol]['quote']['latestPrice'],
#                                                    data[symbol]['quote']['peRatio'],
#                                                    'N/A'
#                                                    ], 
#                                                   index = my_columns), 
#                                         ignore_index = True)
        
    
# final_dataframe

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [17]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:20]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [18]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,410.2400,0.01,N/A
1,APA,30.1700,3.37,N/A
2,BAX,35.2400,6.74,N/A
3,CINF,118.2100,7.87,N/A
4,APTV,82.8400,7.89,N/A
5,CE,159.7100,8.91,N/A
6,CHTR,278.1300,9.00,N/A
7,AFL,86.3000,9.51,N/A
8,CMCSA,39.7000,10.53,N/A
9,ADM,61.1643,10.71,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [19]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [20]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [21]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,410.2400,0.01,1
1,APA,30.1700,3.37,16
2,BAX,35.2400,6.74,14
3,CINF,118.2100,7.87,4
4,APTV,82.8400,7.89,6
5,CE,159.7100,8.91,3
6,CHTR,278.1300,9.00,1
7,AFL,86.3000,9.51,5
8,CMCSA,39.7000,10.53,12
9,ADM,61.1643,10.71,8


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
# symbol = 'AAPL'
# batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(batch_api_call_url).json()

# # P/E Ratio
# pe_ratio = data[symbol]['quote']['peRatio']

# # P/B Ratio
# pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# #P/S Ratio
# ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# # EV/EBITDA
# enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
# ebitda = data[symbol]['advanced-stats']['EBITDA']
# ev_to_ebitda = enterprise_value/ebitda

# # EV/GP
# gross_profit = data[symbol]['advanced-stats']['grossProfit']
# ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [23]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol in stocks['Ticker'][:100]:
    # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'

    outer_data = requests.get(batch_api_call_url).json()
    quote_data = outer_data[0]
    advanced_data = outer_data[1]
    # for symbol in symbol_string.split(','):
    enterprise_value = advanced_data['enterpriseValue']
    ebitda = advanced_data['EBITDA']
    gross_profit = advanced_data['grossProfit']
    
    try:
        ev_to_ebitda = enterprise_value/ebitda
    except TypeError:
        ev_to_ebitda = np.NaN
    
    try:
        ev_to_gross_profit = enterprise_value/gross_profit
    except TypeError:
        ev_to_gross_profit = np.NaN
        
    new_row = pd.Series([
                        symbol,
                        quote_data['latestPrice'],
                        'N/A',
                        advanced_data['peRatio'],
                        'N/A',
                        advanced_data['priceToBook'],
                        'N/A',
                        advanced_data['priceToSales'],
                        'N/A',
                        ev_to_ebitda,
                        'N/A',
                        ev_to_gross_profit,
                        'N/A',
                        'N/A'
                        ],
                        index = rv_columns)

    rv_dataframe = pd.concat([rv_dataframe, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_14588\3346017265.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rv_dataframe = pd.concat([rv_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [24]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.28,N/A,35.868449,N/A,7.16,N/A,6.5800,N/A,28.678782,N/A,13.257093,N/A,N/A
1,AAL,15.19,N/A,19.932025,N/A,-1.81,N/A,0.1874,N/A,6.124304,N/A,0.639395,N/A,N/A
2,AAP,75.64,N/A,151.634787,N/A,1.79,N/A,0.3995,N/A,13.762651,N/A,1.280369,N/A,N/A
3,AAPL,187.43,N/A,28.629302,N/A,38.74,N/A,7.5300,N/A,22.460306,N/A,16.736069,N/A,N/A
4,ABBV,161.59,N/A,47.949349,N/A,35.64,N/A,5.2500,N/A,15.473978,N/A,10.073157,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,118.21,N/A,7.798892,N/A,1.46,N/A,1.7300,N/A,6.034160,N/A,1.728474,N/A,N/A
96,CL,94.57,N/A,29.716245,N/A,337.34,N/A,3.9300,N/A,19.536601,N/A,7.311346,N/A,N/A
97,CLX,138.55,N/A,71.692806,N/A,189.08,N/A,2.3900,N/A,21.850752,N/A,6.470018,N/A,N/A
98,CMA,54.00,N/A,10.718131,N/A,1.27,N/A,1.3500,N/A,7.224734,N/A,1.364794,N/A,N/A


In [25]:
another_refrence_dataframe =rv_dataframe

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [26]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.02,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.50,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,287.92,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
48,ATVI,94.42,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.92,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [27]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_14588\2238724823.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [28]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [29]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0     0.74
1     0.39
2     0.99
3     0.58
4     0.86
      ... 
95    0.08
96    0.62
97    0.96
98    0.14
99    0.13
Name: PE Percentile, Length: 100, dtype: object
0     0.71
1     0.05
2     0.26
3     0.97
4     0.96
      ... 
95    0.17
96     1.0
97    0.99
98    0.14
99    0.29
Name: PB Percentile, Length: 100, dtype: object
0     0.865
1      0.03
2      0.06
3       0.9
4      0.75
      ...  
95     0.36
96     0.67
97    0.425
98     0.28
99     0.26
Name: PS Percentile, Length: 100, dtype: object
0      0.9
1     0.08
2     0.39
3     0.84
4     0.49
      ... 
95    0.07
96    0.73
97    0.82
98    0.14
99    0.11
Name: EV/EBITDA Percentile, Length: 100, dtype: object
0     0.83
1     0.03
2     0.11
3     0.91
4     0.71
      ... 
95    0.17
96    0.58
97    0.46
98    0.12
99    0.21
Name: EV/GP Percentile, Length: 100, dtype: object


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.28,N/A,35.868449,0.74,7.16,0.71,6.5800,0.865,28.678782,0.9,13.257093,0.83,N/A
1,AAL,15.19,N/A,19.932025,0.39,-1.81,0.05,0.1874,0.03,6.124304,0.08,0.639395,0.03,N/A
2,AAP,75.64,N/A,151.634787,0.99,1.79,0.26,0.3995,0.06,13.762651,0.39,1.280369,0.11,N/A
3,AAPL,187.43,N/A,28.629302,0.58,38.74,0.97,7.5300,0.9,22.460306,0.84,16.736069,0.91,N/A
4,ABBV,161.59,N/A,47.949349,0.86,35.64,0.96,5.2500,0.75,15.473978,0.49,10.073157,0.71,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,118.21,N/A,7.798892,0.08,1.46,0.17,1.7300,0.36,6.034160,0.07,1.728474,0.17,N/A
96,CL,94.57,N/A,29.716245,0.62,337.34,1.0,3.9300,0.67,19.536601,0.73,7.311346,0.58,N/A
97,CLX,138.55,N/A,71.692806,0.96,189.08,0.99,2.3900,0.425,21.850752,0.82,6.470018,0.46,N/A
98,CMA,54.00,N/A,10.718131,0.14,1.27,0.14,1.3500,0.28,7.224734,0.14,1.364794,0.12,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [30]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.28,N/A,35.868449,0.74,7.16,0.71,6.5800,0.865,28.678782,0.9,13.257093,0.83,0.809
1,AAL,15.19,N/A,19.932025,0.39,-1.81,0.05,0.1874,0.03,6.124304,0.08,0.639395,0.03,0.116
2,AAP,75.64,N/A,151.634787,0.99,1.79,0.26,0.3995,0.06,13.762651,0.39,1.280369,0.11,0.362
3,AAPL,187.43,N/A,28.629302,0.58,38.74,0.97,7.5300,0.9,22.460306,0.84,16.736069,0.91,0.84
4,ABBV,161.59,N/A,47.949349,0.86,35.64,0.96,5.2500,0.75,15.473978,0.49,10.073157,0.71,0.754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,118.21,N/A,7.798892,0.08,1.46,0.17,1.7300,0.36,6.034160,0.07,1.728474,0.17,0.17
96,CL,94.57,N/A,29.716245,0.62,337.34,1.0,3.9300,0.67,19.536601,0.73,7.311346,0.58,0.72
97,CLX,138.55,N/A,71.692806,0.96,189.08,0.99,2.3900,0.425,21.850752,0.82,6.470018,0.46,0.731
98,CMA,54.00,N/A,10.718131,0.14,1.27,0.14,1.3500,0.28,7.224734,0.14,1.364794,0.12,0.164


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [31]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:20]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [32]:
portfolio_input()

In [33]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,15.1900,32,19.932025,0.39,-1.8100,0.05,0.1874,0.03,6.124304,0.08,0.639395,0.03,0.116
1,AIG,79.4400,6,11.018137,0.15,1.2200,0.13,1.0900,0.185,5.215848,0.04,1.089383,0.09,0.119
2,C,63.5900,7,18.372324,0.34,0.6418,0.06,0.7422,0.13,7.204448,0.13,0.787997,0.06,0.144
3,ALK,43.7800,11,22.674795,0.45,1.4000,0.16,0.5310,0.08,4.494096,0.02,0.558433,0.02,0.146
4,CFG,36.4300,13,12.682768,0.2,0.7622,0.07,1.3300,0.27,7.341177,0.15,1.412318,0.13,0.164
5,CMA,54.0000,9,10.718131,0.14,1.2700,0.14,1.3500,0.28,7.224734,0.14,1.364794,0.12,0.164
6,AIZ,177.3300,2,12.045789,0.18,1.8700,0.28,0.8110,0.15,8.005769,0.17,0.810993,0.07,0.17
7,CINF,118.2100,4,7.798892,0.08,1.4600,0.17,1.7300,0.36,6.034160,0.07,1.728474,0.17,0.17
8,ADM,61.1643,8,10.023662,0.12,1.3100,0.15,0.3325,0.04,7.572489,0.16,5.589675,0.41,0.176
9,BK,58.3500,8,14.080366,0.25,1.2000,0.12,1.2100,0.22,10.313177,0.25,1.208995,0.1,0.188
